In [3]:
import os

In [4]:
with open(os.path.join(os.getcwd(), 'secrets', 'google_api_key.txt')) as f:
    api_key = f.read()

In [5]:
from llama_index.llms.palm import PaLM

llm = PaLM(api_key=api_key)

In [7]:
llm.complete('What is a Large Language model ?')

CompletionResponse(text="A large language model (LLM) is a type of artificial intelligence (AI) model that can understand and generate human language. LLMs are trained on massive datasets of text and code, and they can learn to perform a wide variety of language-related tasks, such as translation, summarization, question answering, and text generation.\n\nLLMs are still under development, but they have the potential to revolutionize a wide range of industries, from customer service to healthcare. For example, LLMs could be used to help businesses create more personalized customer experiences, or to help doctors diagnose and treat diseases.\n\nHowever, LLMs also raise a number of ethical concerns. For example, LLMs could be used to create deepfakes or to spread misinformation. It is important to ensure that LLMs are used responsibly and for the benefit of society.\n\nHere are some of the benefits of using large language models:\n\n* **Improved customer service:** LLMs can be used to hel

In [8]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [14]:
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  hf
)

In [9]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

In [15]:
documents = SimpleDirectoryReader("./tmp/kyc_docs/").load_data()

In [16]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [17]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
query_engine = index.as_query_engine()
response = query_engine.query("What business Amazon is in ?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
print(response)

We serve developers and enterprises of all sizes through AWS, whichoffers a broad set of on-de
mand technology services, including compute, storage, database, analytics, and machine learning, and other services. We also manufactureand sell electronic d
evices. In addition, we provide advertising services to sellers, vendors, publishers, authors, and others, through programs such as sponsored ads,display, and video advertising.


In [20]:
while True:
  query=input()
  if query == 'exit':
    break
  response = query_engine.query(query)
  print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

AMZN


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Delaware


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Amazon 10-K filing contains the company's financial statements and supplementary data.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

As of December 31, 2021, Amazon had cash, cash equivalents, and marketable securities of $84,396 million, compared to $84,653 million as of December 31, 2020.
